<a href="https://colab.research.google.com/github/parthpendal/Concepts---Data-Analysis-and-Machine-Learning/blob/main/WebScrapping_Top_10_Holdings_by_Mutual_funds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [157]:
source=requests.get("https://www.moneycontrol.com/mutual-funds/nav/axis-bluechip-fund-regular-plan/MAA009")
Soup=BeautifulSoup(source.text,'html.parser')
print(Soup.text)





  

Axis Bluechip Fund - Growth [47.9]  | Axis Mutual Fund - Moneycontrol







     

       

 


    {
        "@context": "https://schema.org",
        "@type": "WebPage",
        "name": "Axis Bluechip Fund - Growth [47.9]  | Axis Mutual Fund - Moneycontrol",
        "description": "Get latest NAV, Returns, SIP Returns, Performance, Ranks, Dividends, Portfolio, CRISIL Rank, Expert Recommendations, and Comparison with gold, stock,ULIP etc. Calculate SIP, VIP Returns. Now invest in Axis Bluechip Fund at Moneycontrol.com.",
        "url": "https://www.moneycontrol.com/mutual-funds/nav/axis-bluechip-fund-regular-plan/MAA009",
        "publisher": {
            "@type": "Organization",
            "name": "Moneycontrol",
            "url": "https://www.moneycontrol.com/",
            "logo": {
                "@type": "ImageObject",
                "contentUrl": "https://images.moneycontrol.com/images/common/mc_logo_200x200.png"
            }
        }
    }




    {
        "@co

In [158]:

def get_top10_holding(url):
  source=requests.get(url)
  source.raise_for_status()
  Soup=BeautifulSoup(source.text,'html.parser')
  
  #Get required table information
  Info=Soup.find('section', class_="clearfix section_six graybgtbl").find("div",class_="data_container table-responsive")
  
  #Get table with Header
  stock_invested_in=Info.find_all("tr")
  #print(stock_invested_in)
  
  #Get Headers for stocks in mutual fund portfolio (top 10 holdings)
  Header_list=[]
  Headers=stock_invested_in[0].find_all('th')
  for header in Headers:
    Header_list.append(header.text)
  #print(Header_list)
  
  
  #Get content of the table
  data_list=list()
  stock_invested_in=stock_invested_in[1:]
  for row_entry in stock_invested_in:
    rows=row_entry.find_all("td")
    element_list=[]
    for single_elelemt in rows:
      
      if '\n' in single_elelemt.text:
        element_list.append(single_elelemt.text.split('\n')[2].strip())
      else:
        element_list.append(single_elelemt.text.strip())
    data_list.append(element_list)
  #print(data_list)

  #Making dataframe
  top_10_holding=pd.DataFrame(data_list)
  top_10_holding.columns=Header_list
  return top_10_holding

In [159]:
axis_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/axis-bluechip-fund-regular-plan/MAA009")
ICICI_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/icici-prudential-bluechip-fund/MPI392")
SBI_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/sbi-blue-chip-fund/MSB079")
LNT_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/l-t-india-large-cap-fund/MFF027")
Tata_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/tata-large-cap-fund-regular-plan/MTA006")
Edlweiss_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/edelweiss-large-cap-fund-a/MEW041")
Kotak_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/kotak-bluechip-fund-regular-plan/MKM003")
Invesco_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/invesco-india-largecap-fund/MLI339")
Aditya_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/aditya-birla-sun-life-frontline-equity-fund/MAC031")
Franklin_large_cap_top_10_holdings=get_top10_holding("https://www.moneycontrol.com/mutual-funds/nav/franklin-india-bluechip-fund-growth/MKP001")

In [160]:
fund_list={"Axis":axis_large_cap_top_10_holdings,"ICICI":ICICI_large_cap_top_10_holdings,"SBI":SBI_large_cap_top_10_holdings,
                         "LNT":LNT_large_cap_top_10_holdings,"Tata":Tata_large_cap_top_10_holdings,"Edlweiss":Edlweiss_large_cap_top_10_holdings,
           "Kotak":Kotak_large_cap_top_10_holdings,"Invesco":Invesco_large_cap_top_10_holdings,"Aditya":Aditya_large_cap_top_10_holdings,"Franklin":Franklin_large_cap_top_10_holdings}



In [161]:
axis_large_cap_top_10_holdings.head()

,Stock Invested in,Sector,Value(Mn),% of Total Holdings,1M Change,1Y Highest Holding,1Y Lowest Holding,Quantity,1M Change in Qty
0,Infosys Ltd.,Computers - software,34545.8,9.99%,0.00%,9.79% (Jun 2021),8.59% (Oct 2021),1.83 Cr,8.00 L
1,Bajaj Finance Ltd.,Nbfc,30742.8,8.89%,0.00%,10.19% (Aug 2021),8.61% (May 2021),44.06 L,44.87 k
2,ICICI Bank Ltd.,Banks,29583.8,8.55%,0.00%,9.27% (Oct 2021),6.07% (Dec 2020),4.00 Cr,-4.69 L
3,Avenue Supermarts Ltd.,Retailing,27415.8,7.93%,0.00%,7.84% (Nov 2021),5.4% (Jan 2021),58.69 L,2.87 L
4,Tata Consultancy Services Ltd.,Computers - software,25626.4,7.41%,0.00%,8.03% (Aug 2021),5.84% (Feb 2021),68.55 L,2.20 L


In [165]:
#Getting list of stocks
list_of_stocks=pd.DataFrame()
list_of_stocks=pd.concat(fund_list.values())
list_of_stocks=list_of_stocks['Stock Invested in'].drop_duplicates()
list_of_stocks=pd.DataFrame(list_of_stocks)
my_dataframe=list_of_stocks.copy()

def get_average_values (my_dataframe,Parameter_to_be_compared):
  #Getting holdings for each stocks
  for x,y in zip(fund_list.values(),fund_list.keys()):
    x=x[['Stock Invested in',Parameter_to_be_compared]]
    x.columns=['Stock Invested in',y]
    my_dataframe=pd.merge(my_dataframe,x,on='Stock Invested in',how="left")
  my_dataframe=my_dataframe.replace(np.NaN,"0")

  for column in fund_list.keys():
    my_dataframe[column]=pd.DataFrame(x.split("%")[0] if "%" in x else x for x in my_dataframe[column] )
    my_dataframe[column]=my_dataframe[column].astype("float64")
  
  my_dataframe[Parameter_to_be_compared]=my_dataframe[fund_list.keys()].mean(axis=1)
  my_dataframe=my_dataframe.sort_values(Parameter_to_be_compared,ascending=False)
  #print(my_dataframe.head(10))
  return my_dataframe



In [167]:
Lowest_value_dataframe=get_average_values(my_dataframe,'1Y Lowest Holding')
Highest_value_dataframe=get_average_values(my_dataframe,'1Y Highest Holding')
Current_value_dataframe=get_average_values(my_dataframe,'% of Total Holdings')

#Get overall Summary
Final_dataframe=my_dataframe.copy()
for x,y in zip([Lowest_value_dataframe,Highest_value_dataframe,Current_value_dataframe],['1Y Lowest Holding','1Y Highest Holding','% of Total Holdings']):
    Final_dataframe=pd.merge(Final_dataframe,x[['Stock Invested in',y]],on='Stock Invested in',how="left")
Final_dataframe.sort_values('% of Total Holdings', ascending=False)

,Stock Invested in,1Y Lowest Holding,1Y Highest Holding,% of Total Holdings
2,ICICI Bank Ltd.,7.790,9.317,8.399
0,Infosys Ltd.,6.731,8.422,7.767
5,HDFC Bank Ltd.,7.188,9.109,7.324
6,Reliance Industries Ltd.,4.163,6.267,5.504
10,Larsen & Toubro Ltd.,2.285,3.783,3.839
12,Bharti Airtel Ltd.,2.384,3.437,2.788
16,State Bank Of India,1.888,3.672,2.701
4,Tata Consultancy Services Ltd.,1.527,2.641,2.470
11,Axis Bank Ltd.,2.037,2.702,2.266
14,HCL Technologies Limited,0.971,2.027,1.948


In [168]:
#Get fundwise holdings
Current_value_dataframe


,Stock Invested in,Axis,ICICI,SBI,LNT,Tata,Edlweiss,Kotak,Invesco,Aditya,Franklin,% of Total Holdings
2,ICICI Bank Ltd.,8.55,9.54,7.09,8.59,8.89,7.21,7.04,9.01,9.39,8.68,8.399
0,Infosys Ltd.,9.99,7.34,6.44,7.62,7.61,7.76,7.37,9.13,8.52,5.89,7.767
5,HDFC Bank Ltd.,7.34,7.84,8.17,7.50,6.67,6.28,6.79,9.18,7.10,6.37,7.324
6,Reliance Industries Ltd.,3.97,6.18,4.34,7.80,6.41,6.52,7.55,7.49,4.78,0.00,5.504
10,Larsen & Toubro Ltd.,0.00,5.42,4.51,3.40,3.38,2.70,3.37,5.38,4.32,5.91,3.839
12,Bharti Airtel Ltd.,0.00,5.03,0.00,3.00,3.37,4.86,0.00,3.55,4.01,4.06,2.788
16,State Bank Of India,0.00,0.00,2.66,4.56,5.57,3.58,0.00,3.68,2.86,4.10,2.701
4,Tata Consultancy Services Ltd.,7.41,0.00,0.00,3.65,4.19,0.00,4.25,0.00,0.00,5.20,2.470
11,Axis Bank Ltd.,0.00,5.15,0.00,0.00,3.59,3.34,3.07,4.22,3.29,0.00,2.266
14,HCL Technologies Limited,0.00,3.26,4.63,0.00,0.00,2.63,0.00,0.00,3.08,5.88,1.948
